In [ ]:
# Set system path to allow imports
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from ephys import ephys
from session_utils import find_all_sessions
from position_analysis import *


# Load all session names and paths as dict
session_list = find_all_sessions(sheet_path = 'https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0',
                                 data_path = '/home/isabella/Documents/isabella/jake/recording_data',
                                 sorting_suffix = 'sorting_ks2_custom')


# Constants
max_speed = 5  # m/s
smoothing_window_size = 3  # Choose an appropriate window size for smoothing

In [ ]:
processed_posdata = []

# Loop through sessions
trials_loaded = 0
for session, path in session_list.items():
    
    obj = ephys(recording_type = 'nexus', path = path)
    
    # Load position for t-maze trials
    for trial_iterator, trial in enumerate(obj.trial_list):
        if 't-maze' in trial:
            obj.load_pos(trial_iterator, reload_flag = False) #Needs reloading each time for now
            trials_loaded +=1
            
            # Extract XY position data and calculate performance using funcs from position_analysis.py
            xy_pos = obj.pos_data[trial_iterator]['xy_position']
            performance = calculate_choices(xy_pos, assign_sectors(xy_pos.T, pos_header = obj.pos_data[trial_iterator]['header']))
            
            # Add age and animal ID
            obj.pos_data[trial_iterator]['age'] = obj.age
            obj.pos_data[trial_iterator]['animal'] = obj.animal
            
            # Add processed position data to array
            processed_posdata.append({'xy_position': xy_pos,
                                      'age': obj.age,
                                      'animal': obj.animal,
                                      'performance': performance})
                                      

            
print(f'{trials_loaded} t-maze trials loaded')

# Convert processed_posdata to DataFrame
data_df = pd.DataFrame(processed_posdata)

#Extract performance metrics for plotting
data_df['total_choices'] = data_df['performance'].apply(lambda x: x['total_choices'])
data_df['p_correct'] = data_df['performance'].apply(lambda x: x['p_correct'])

### Plotting:


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

# Remove NaN values
df_clean = data_df.dropna(subset=['age', 'total_choices', 'animal', 'p_correct'])
df_clean['age'] = df_clean['age'].astype(int)


# Plot Total Choices vs Age

# Calculate the correlation coefficient
corr, p_val = pearsonr(df_clean['age'], df_clean['total_choices'])

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='age', y='total_choices', hue='animal', palette="deep", s = 100)

# If correlation is significant, add a best fit line
if p_val < 0.05:
    sns.regplot(data=df_clean, x='age', y='total_choices', scatter=False, color='black')

plt.xlabel('Age (Days)', fontsize = 15)
plt.ylabel('Total Choices', fontsize = 15)
plt.title(f"Total Choices vs Age (Pearson r = {corr:.2f}, p = {p_val:.3f})", fontsize = 20)
plt.tight_layout()
plt.savefig('/home/isabella/Documents/isabella/jake/ephys_analysis/figures/t-maze_performance/age_vs_total_choices.png')
plt.show()


In [ ]:
# Plot Age vs Proportion of Correct Choices

# Create a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='age', y='p_correct', hue='animal', palette="deep", s = 100)

# Calculate the correlation coefficient
corr, p_val = pearsonr(df_clean['age'], df_clean['p_correct'])

# If correlation is significant, add a best fit line
if p_val < 0.05:
    sns.regplot(data=df_clean, x='age', y='p_correct', scatter=False, color='black')

plt.xlabel('Age (Days)', fontsize = 15)
plt.ylabel('Proportion Correct Choices', fontsize = 15)
plt.title(f"Performance vs Age (Pearson r = {corr:.2f}, p = {p_val:.5f})", fontsize = 20)
plt.tight_layout()
plt.savefig('/home/isabella/Documents/isabella/jake/ephys_analysis/figures/t-maze_performance/age_vs_p_correct.png')
plt.show()


In [ ]:


X = xy_pos.iloc[0,:]
Y = xy_pos.iloc[1,:]
angles = direction_disp  # Random angles in radians

# Create masks for valid (non-nan) X, Y, angles, and speed data
mask = ~np.isnan(X) & ~np.isnan(Y) & ~np.isnan(angles) & ~np.isnan(speed)

# Compute arrow directions, scaled by speed
U = speed[mask] * np.cos(angles[mask])
V = speed[mask] * np.sin(angles[mask])

plt.figure()
#Plot every 10th point for visibility
plt.quiver(X[mask][::10], Y[mask][::10], U[::10], V[::10], angles='xy', scale_units='xy', scale=20)
plt.show()

In [ ]:
# Define the reward and start sectors
reward_sectors = [1, 9]
start_sector = 8

sector_numbers = assign_sectors(xy_pos.T)
plt.scatter(xy_pos.iloc[0,:], xy_pos.iloc[1,:], c = sector_numbers)
plt.colorbar()
plt.show()